In [2]:
import nltk
import numpy as np 
import pandas as pd
import re
import glob
import os
import shutil
from collections import Counter
import random

In [35]:
def clean_text(text):
    """
    Remove leading bullets or numbered indexes (1., 2., ..., 11.) from lines.
    """
    cleaned_lines = []
    for line in text.split("\n"):
        cleaned_line = re.sub(r'^\s*\d{1,2}[\.\)]\s*', '', line, flags=re.MULTILINE)
        cleaned_lines.append(cleaned_line)
    return "\n".join(cleaned_lines)

def read_files_in_dirs(file_path = 'dataset'):
    """
    Args: 
        file_path : name of the file in which the txt files are
    Reads files in a folder preprocesses it and puts it in a new folder named cleaned_dataset
    removes trailing spaces and beginning digits.  
    Returns:
        None
        
    """
    base_path = r'D:\Bravo Six\Fund of NLP\Assignment 3'+'\\'+f'{file_path}'
    txtFilePaths = os.listdir(base_path)
    cleaned_path = os.path.join(base_path, 'cleaned_dataset')
    if os.path.exists(cleaned_path):
        shutil.rmtree(cleaned_path)
    for file in txtFilePaths:
        fullFilePath = base_path+'\\'+file
        cleaned_file = open(fullFilePath, 'r', encoding='utf-8',errors='replace')
        cleaned_file = cleaned_file.read()
        cleanedFileTxt = clean_text(cleaned_file)
        cleanedFileInNewDir(cleanedFileTxt, file)
        print('Cleaned File: ', file)
        
def cleanedFileInNewDir(fileText : str =''  ,file_to_put : str = '',folder_path : str= 'cleaned_dataset'):
    """
    Helper function that makes a new directory and puts processed text in a new file 
    Args:
        fileText : processed text from the raw file
        file_to_put : name of the file to put in the new folder e.g file1.txt
        file_path : a folder name in which cleaned files will be placed
    Returns:
        None
    """
    base_path = r'D:\Bravo Six\Fund of NLP\Assignment 3'+'\\'+f'{folder_path}'
    os.makedirs(base_path,exist_ok=True)
    file = os.path.join(base_path, file_to_put)
    with open(file, 'w', encoding='utf-8') as f:
        f.write(fileText)
        f.close()
    
    
    
    


In [36]:
def readCleanedText(folder_path='cleaned_dataset') -> list:
    """
    Reads all files from a folder
    Args:
        folder_path : folder from which to read files 
    Returns:
        list of strings 

    """
    dirPath = r'D:\Bravo Six\Fund of NLP\Assignment 3' + '\\' + f'{folder_path}'
    cleanedFiles = os.listdir(dirPath)
    cleanedText = []
    for file in cleanedFiles:
        fullFilePath = os.path.join(dirPath, file)
        with open(fullFilePath, 'r', encoding='utf-8', errors='replace') as f:
            cleanedText.extend(f.read().splitlines())  
    return cleanedText  
def readFirstToken(file_path='cleaned_dataset'):  # For bonus task only
    dirPath = r'D:\Bravo Six\Fund of NLP\Assignment 3' + '\\' + f'{file_path}'
    cleanedFiles = os.listdir(dirPath)
    first_tokens = []
    for file in cleanedFiles:
        fullFilePath = os.path.join(dirPath, file)
        with open(fullFilePath, 'r', encoding='utf-8', errors='replace') as f:
            for line in f:
                words = line.split()
                if words:  # Ensure line is not empty
                    first_word = words[0]
                    if len(first_word) > 1 and not first_word[0].isdigit():
                        first_tokens.append(first_word)

    return first_tokens
def getTokens(lines : list):
    """
    Returns individual words/tokens.
    Args: 
        lines : list of strings 
    Returns:
        Individual tokens in a list 
    Example:
        >>>getTokens(["My name is hello world"])
        ["My", "name", "is", "hello", "world"]
    """
    return [tok for line in lines for tok in line.split()]
    
lines = readCleanedText()
print(lines)
tokens = getTokens(lines)
print(tokens)
len(tokens)

['subha 5 bjhey uthna perha trip thaa jaldi jaldi ready hua aur 0540 ghar saay nikal gaay', '0615 bus chal perhee joo kaay first time thaa kaay trip time saay chala', 'nust kaay 3 larkay thy unsaay batain kee phir bluetooth trip coordinator kaay pass thaa tou humm shoor daltay rahay kaay song change krr dain', 'phir murree mein nashtay kaay leya uthay phir usskay baad bluetooth humain mil gaya ', 'mein tou soogaya phir hum jaga prr phouch gaay udhar 3 peaks theen 2 perr char gaay', 'aik prr sahi ghalat raastay saay gaay thy full steep thaa. mujhey tou laga mein gaya', 'peak prr pohouch gaya phir picks leen ', 'phir slow wala group bhee aa gaya thaa, unn kaay sath dubara picks leen', 'wapsi prr jeep mein bohot rash thaa', 'phir dinner keya ghar gaay aur soogaau', 'Subah 8 bjy utha. Fresh hua nashta kiya aur university k liye tyaar honay laga.', '9 bjy apne bike pe university k liye nikal gaya. Aj friday ha aur aj mere sirf 2 classes huti hain.', 'University ponch k classes li. pehla lec

16294

### Unigram Model

In [37]:
def train_unigram(tokens : list) ->dict:
    """
    Returns a dictionary where the keys are the unique individual tokens and values are their respective probabilities
    calculated using the formula: 
        P(word) = wordCount / totalWords
    Args:
        tokens (list): A list of individual tokens (words).
    Returns:
        dict: A dictionary where keys are unique words and values are their probabilities.
    Example:
        >>> train_unigram(["my", "name", "is", "hi"])
        {'my': 0.25, 'name': 0.25, 'is': 0.25, 'hi': 0.25}

        >>> train_unigram(["a", "a", "b", "b", "b"])
        {'a': 0.4, 'b': 0.6}
    """
    wordCount = Counter(tokens)
    # print(len(wordCount))
    # print(wordCount)
    wordProbs = wordCount.copy()
    for key in wordCount.keys():
        wordProbs[key] =(wordCount[key] / len(tokens))
    
    return dict(wordProbs)
    
    
def predictUnigram(unigramProbs : dict) -> str:
    """
    Generates a random sentence using a unigram model.

    The function starts with a randomly chosen first word from `readFirstToken()` 
    and then selects the remaining words randomly from the unigram probability dictionary.
    The sentence length is randomly chosen between 7 and 12 words.

    Args:
        unigramProbs (dict): A dictionary where keys are words and values are their probabilities.

    Returns:
        str: A randomly generated sentence based on unigram probabilities.

    Example:
        >>> unigramProbs = {'hello': 0.2, 'world': 0.3, 'AI': 0.5}
        >>> predictUnigram(unigramProbs)
        'AI world hello AI world AI hello'

    Notes:
        - Words are chosen randomly, so the output may differ each time.
        - The sentence length is randomly chosen between 7 and 12 words.
        - Once a word is selected, it is removed from the available choices.

    """
    finalSentence = random.choice(readFirstToken()) + " "
    lengthOfSentence = np.random.randint(7,12)
    copyUgProbs = unigramProbs.copy()
    for i in range(lengthOfSentence):
        nextWord = np.random.choice(list(copyUgProbs.keys()))
        finalSentence += nextWord + " "
        del copyUgProbs[nextWord]
    # print(len(copyUgProbs))
    return finalSentence
        
    
    
    
        
        
unigramProbs = train_unigram(tokens)
print(predictUnigram(unigramProbs))
print(len(unigramProbs))

    

Aaj chala mai. movie usse two successful bja prepare 
3138


### Bigram Model

In [51]:
def train_bigram(bigrams : list) -> dict:
    """
    Computes bigram probabilities using Laplace (Add-1) smoothing.

    The function calculates the probability of a word occurring after another word
    using the formula:

        P(w2 | w1) = log((count(w1, w2) + 1) / (count(w1) + V))

    where:
        - count(w1, w2) is the number of times the bigram (w1, w2) appears.
        - count(w1) is the number of times w1 appears as a unigram.
        - V is the vocabulary size (number of unique words).
        - log is applied to avoid underflow.

    Args:
        bigrams (list): A list of bigrams (tuples of two words).

    Returns:
        dict: A dictionary where keys are bigrams (tuples) and values are their log probabilities.

    Example:
        >>> train_bigram([("hello", "world"), ("hello", "there"), ("world", "is")])
        {("hello", "world"): -1.2, ("hello", "there"): -1.3, ("world", "is"): -0.9}
    """
    tokens = getTokens(readCleanedText())
    wordCount = dict(Counter(tokens))
    bigramFreqs = Counter(bigrams)
    bigramProbs = dict(bigramFreqs.copy())
    V = len(wordCount)
    for key in bigramProbs.keys():
        bigramProbs[key] = (np.log((bigramProbs[key] +1) /( wordCount.get(key[0],0 ) + V)))
    return dict(bigramProbs)
    


def predictBigram(bigramProbs :dict) -> tuple:
    """
    Generates a sentence using a bigram model.

    The function starts with a randomly chosen word from `readFirstToken()`
    and generates the next words based on bigram probabilities.
    It returns two versions of the sentence:
    1. **Maximum Probability Sentence:** Picks the most probable word at each step.
    2. **Random Probability Sentence:** Picks the next word randomly based on bigram probabilities.

    Args:
        bigramProbs (dict): A dictionary where keys are bigram tuples, and values are log probabilities.

    Returns:
        tuple: A tuple containing:
            - str: The sentence generated using maximum probability words.
            - str: The sentence generated using random sampling.

    Example:
        >>> bigramProbs = {("hello", "world"): -1.2, ("world", "is"): -0.9, ("is", "good"): -1.0}
        >>> predictBigram(bigramProbs)
        ('hello world is good', 'hello world is good')
    """
    first_word = np.random.choice(readFirstToken())
    while first_word not in [key[0] for key in bigramProbs.keys()]:
        first_word = np.random.choice(readFirstToken())

    finalSentenceRand = first_word + " "
    finalSentenceMax = first_word + " "
    current_word = first_word
    lengthOfSentence = np.random.randint(7,12)
    copyBgProbs = bigramProbs.copy()
    for _ in range(lengthOfSentence):
        next_word_candidates = {key[1]: prob for key, prob in bigramProbs.items() if key[0] == current_word}
        # print(next_word_candidates)
        if not next_word_candidates:
            break  
        next_words = list(next_word_candidates.keys())
        probabilities = np.exp(list(next_word_candidates.values())) 
        probabilities /= probabilities.sum()
        
        next_word = np.random.choice(next_words, p=probabilities)
        finalSentenceRand += next_word + " "
        
        max_word = max(next_word_candidates, key=next_word_candidates.get)  # Pick the word with max probability
        finalSentenceMax += max_word + " "
        current_word = next_word  

    return finalSentenceMax.strip() , finalSentenceRand.strip() 
    

def bigrams(sentences : list, reverse :bool = False):   
    """
    Generates a list of bigrams from a list of sentences.

    A bigram is a pair of consecutive words (w1, w2) from a sentence.
    If `reverse` is set to `True`, the function processes sentences in reverse order.

    Args:
        sentences (list): A list of sentences, where each sentence is a string.
        reverse (bool, optional): If True, processes sentences in reverse order. Defaults to False.

    Returns:
        list: A list of bigrams (tuples of two words).

    Example:
        >>> bigrams(["hello world", "how are you"])
        [('hello', 'world'), ('how', 'are'), ('are', 'you')]

        >>> bigrams(["hello world", "how are you"], reverse=True)
        [('world', 'hello'), ('you', 'are'), ('are', 'how')]
    """
    splitSenteces = [sentence.split() for sentence in sentences]
    bigrams = []
    if reverse:
        for sentence in splitSenteces:
            sentence = list(reversed(sentence))
            # print(sentence)
            for i in range(len(sentence)-1):
                bigrams.append((sentence[i],sentence[i+1]))
    else:
        for sentence in splitSenteces:
            for i in range(len(sentence)-1):
                bigrams.append((sentence[i],sentence[i+1]))
        print(len(bigrams))
    
    return bigrams

bigrams(lines,1)
train_bigram(bigrams=bigrams(lines))
predictBigram(train_bigram(bigrams(lines,0)))

15368
15368


('Annatummy restaurant gaye. Or ghar ne or phir ki',
 'Annatummy restaurant gaye. Phir Khadija stickers or baatain ki.')

### Bidirectional

In [40]:
def train_bidirectional_bigrams(lines):
    """
    Trains both a forward bigram model and a backward bigram model.

    The function generates bigram probabilities in both:
    - **Forward Direction:** Normal left-to-right bigrams.
    - **Backward Direction:** Reverse bigrams (right-to-left).

    Args:
        lines (list): A list of sentences (strings).

    Returns:
        tuple: A tuple containing:
            - dict: Forward bigram probabilities.
            - dict: Backward bigram probabilities.

    Example:
        >>> lines = ["hello world", "how are you"]
        >>> forward_probs, backward_probs = train_bidirectional_bigrams(lines)
        >>> forward_probs
        {("hello", "world"): -1.2, ("how", "are"): -0.9, ("are", "you"): -1.0}
        >>> backward_probs
        {("world", "hello"): -1.3, ("you", "are"): -1.1, ("are", "how"): -0.8}
    """
    forward_bigrams = bigrams(lines, reverse=False)  # Normal bigrams
    backward_bigrams = bigrams(lines, reverse=True)  # Reverse bigrams

    forward_probs = train_bigram(forward_bigrams)  # Train forward model
    backward_probs = train_bigram(backward_bigrams)  # Train backward model

    return forward_probs, backward_probs


def predictBidirectionalBigram(forward_probs: dict, backward_probs: dict):
    """
    Generates a sentence using both forward and backward bigram probabilities.

    The function starts with a randomly selected word and predicts the next word based
    on a combination of:
    - **Forward bigram probabilities:** Predicts the next word.
    - **Backward bigram probabilities:** Predicts the previous word.

    The two probability distributions are combined by multiplying their probabilities.

    Args:
        forward_probs (dict): A dictionary containing forward bigram probabilities.
        backward_probs (dict): A dictionary containing backward bigram probabilities.

    Returns:
        str: A generated sentence using bidirectional bigram probabilities.

    Example:
        >>> forward_probs = {("hello", "world"): -1.2, ("how", "are"): -0.9, ("are", "you"): -1.0}
        >>> backward_probs = {("world", "hello"): -1.3, ("you", "are"): -1.1, ("are", "how"): -0.8}
        >>> predictBidirectionalBigram(forward_probs, backward_probs)
        'hello world is great'
    """
    first_word = np.random.choice(readFirstToken())
    while first_word not in [key[0] for key in forward_probs.keys()]:
        first_word = np.random.choice(readFirstToken())

    finalSentence = first_word + " "
    current_word = first_word
    lengthOfSentence = np.random.randint(7, 12)

    for _ in range(lengthOfSentence):
        forward_candidates = {key[1]: prob for key, prob in forward_probs.items() if key[0] == current_word}
        backward_candidates = {key[0]: prob for key, prob in backward_probs.items() if key[1] == current_word}
        
        combined_candidates = {}
        for word in set(forward_candidates.keys()).union(set(backward_candidates.keys())):
            probForward = np.exp(forward_candidates.get(word, -np.inf))  
            probBackward = np.exp(backward_candidates.get(word, -np.inf))  
            combined_candidates[word] = probForward * probBackward  

        if not combined_candidates:
            break

        next_word = max(combined_candidates, key=combined_candidates.get) 
        finalSentence += next_word + " "
        current_word = next_word  

    return finalSentence.strip()

forward_probs, backward_probs = train_bidirectional_bigrams(lines) 
bidirectional_sentence = predictBidirectionalBigram(forward_probs, backward_probs)  

print("Bidirectional Sentence:", bidirectional_sentence)

15368
Bidirectional Sentence: Ammi Abbu se free hukr university ka kaam


### Trigram Model

In [43]:
def train_trigram(trigrams : list, smoothing : str = 'laplace') -> dict:
    """
    Computes trigram probabilities using different smoothing techniques.

    The function calculates the probability of a word occurring after a bigram 
    using different smoothing techniques:

    - **Laplace Smoothing (Add-1):**
      P(w3 | w1, w2) = log((count(w1, w2, w3) + 1) / (count(w1, w2) + V))

    - **Kneser-Ney Smoothing:**
      P_KN(w3 | w1, w2) = log((max(count(w1, w2, w3) - D, 0) / count(w1, w2)) 
      + λ * (count(w2, w3) / V))

    - **Backoff Smoothing:**
      If trigram probability is unavailable, backs off to bigram and unigram models.

    Args:
        trigrams (list): A list of trigrams (tuples of three words).
        smoothing (str, optional): The smoothing technique to use ('laplace', 'kneser-ney', or 'backoff'). 
                                   Defaults to 'laplace'.

    Returns:
        dict: A dictionary where keys are trigrams (tuples) and values are log probabilities.

    Example:
        >>> train_trigram([("hello", "world", "today"), ("hello", "world", "again")], smoothing="laplace")
        {("hello", "world", "today"): -1.2, ("hello", "world", "again"): -1.3}
    """
    tokens = getTokens(readCleanedText())
    bigramFreqs = Counter(bigrams(readCleanedText()))
    unigramFreqs = Counter(getTokens(readCleanedText()))
    trigramFreqs = Counter(trigrams)
    D = 0.75  
    V = len(set(getTokens(readCleanedText())))
    V = len(bigramFreqs)
    # print(len(unigramFreqs),len(bigramFreqs),len(trigramFreqs))
    trigramProbs = {}
    for key in trigramFreqs.keys():
        bigram = (key[0], key[1])

        # Get counts
        trigram_count = trigramFreqs[key]
        bigram_count = bigramFreqs.get(bigram, 0)
        unigram_count = unigramFreqs.get(key[1], 0)

        if smoothing == "laplace":
            trigramProbs[key] = np.log((trigram_count + 1) / (bigram_count + V))

        elif smoothing == "kneser-ney":
            lambda_factor = (D / bigram_count) if bigram_count > 0 else 0
            trigramProbs[key] = np.log(max(trigram_count - D, 0) / bigram_count + lambda_factor * (bigramFreqs.get(bigram, 1) / V))

        elif smoothing == "backoff":
            alpha, beta = 0.4, 0.6  
            if bigram_count > 0:
                trigramProbs[key] = np.log(alpha * (trigram_count / bigram_count) + beta * (unigram_count / len(tokens)))
            else:
                trigramProbs[key] = np.log(unigram_count / len(tokens))

    return trigramProbs
    



def predictTrigram(trigramProbs: dict) -> tuple:
    """
    Generates a sentence using a trigram model.

    The function starts with a randomly selected bigram from `trigramProbs`
    and generates the remaining words based on trigram probabilities.
    
    Two versions of the sentence are returned:
    1. **Maximum Probability Sentence:** Selects the most probable next word at each step.
    2. **Random Probability Sentence:** Selects the next word randomly based on trigram probabilities.

    Args:
        trigramProbs (dict): A dictionary where keys are trigram tuples, and values are log probabilities.

    Returns:
        tuple: A tuple containing:
            - str: The sentence generated using maximum probability words.
            - str: The sentence generated using random sampling.

    Example:
        >>> trigramProbs = {("hello", "world", "today"): -1.2, ("hello", "world", "again"): -1.3}
        >>> predictTrigram(trigramProbs)
        ('hello world today is great', 'hello world again AI project')
    """
    if not trigramProbs:
        raise ValueError("Trigram probabilities dictionary is empty.")
    first_bigram = random.choice(list(trigramProbs.keys()))[:2]  
    finalSentenceRand = " ".join(first_bigram)  
    finalSentenceMax = " ".join(first_bigram)  
    current_bigram = first_bigram  

    lengthOfSentence = np.random.randint(7, 12)
    # print(lengthOfSentence)
    for _ in range(lengthOfSentence - 2):  # -2 because we already have two words
        next_word_candidates = {key[2]: prob for key, prob in trigramProbs.items() if key[:2] == current_bigram}

        if not next_word_candidates:  
            current_bigram = random.choice(list(trigramProbs.keys()))[:2]
            finalSentenceRand += " " + current_bigram[1]
            finalSentenceMax += " " + current_bigram[1]
            continue  

        next_words = list(next_word_candidates.keys())
        probabilities = np.exp(list(next_word_candidates.values()))
        probabilities /= probabilities.sum()

        next_word = np.random.choice(next_words, p=probabilities)
        finalSentenceRand += " " + next_word  

        max_word = max(next_word_candidates, key=next_word_candidates.get)
        finalSentenceMax += " " + max_word  

        current_bigram = (current_bigram[1], next_word)  

    return finalSentenceMax.strip(), finalSentenceRand.strip()
 
    

def trigrams(sentences : list, reverse :bool = False) ->list:   
    """
    Generates a list of trigrams from a list of sentences.

    A trigram is a sequence of three consecutive words (w1, w2, w3) in a sentence.
    If `reverse` is set to `True`, the function processes sentences in reverse order.

    Args:
        sentences (list): A list of sentences, where each sentence is a string.
        reverse (bool, optional): If True, processes sentences in reverse order. Defaults to False.

    Returns:
        list: A list of trigrams (tuples of three words).

    Example:
        >>> trigrams(["hello world today", "how are you"])
        [('hello', 'world', 'today'), ('how', 'are', 'you')]

        >>> trigrams(["hello world today", "how are you"], reverse=True)
        [('today', 'world', 'hello'), ('you', 'are', 'how')]
    """
    splitSenteces = [sentence.split() for sentence in sentences]
    trigrams = []
    if reverse:
        for sentence in splitSenteces:
            sentence = list(reversed(sentence))
            # print(sentence)
            for i in range(len(sentence)-2):
                trigrams.append((sentence[i],sentence[i+1],sentence[i+2]))
    else:
        for sentence in splitSenteces:
            for i in range(len(sentence)-2):
                trigrams.append((sentence[i],sentence[i+1],sentence[i+2]))
                # print(trigrams)
        # print(len(trigrams))
    
    return trigrams

trigrams(lines)
train_trigram(trigrams=trigrams(lines))
predictTrigram(train_trigram(trigrams=trigrams(lines)))

15368
15368


('idhar udhar ki baatein ki. phir maine',
 'idhar udhar ki baatein ki. phir maine')

## Generating Diaries


In [53]:

def writeDiaryToFile(var_name, diary):
    with open(f"{var_name}.txt", "w", encoding="utf-8") as f:
        for sentence in diary:
            f.write(sentence + "\n")


In [55]:
lines = readCleanedText()
tokens= getTokens(lines)
unigramProbs=train_unigram(tokens)
unigramDiary = [predictUnigram(unigramProbs) for _ in range(10)]
bi_grams = bigrams(lines)
bigramProbs = train_bigram(bi_grams)
bigramDiary = [predictBigram(bigramProbs)[0] for _ in range(10)]
revBigrams = bigrams(lines,1)
revBigramProbs = train_bigram(revBigrams)
revBigramDiary = [predictBigram(revBigramProbs)[0] for _ in range(10)]
straightBigrams, reverseBigrams = train_bidirectional_bigrams(lines)
biBigramDiary = [predictBidirectionalBigram(straightBigrams,reverseBigrams) for _ in range(10)]
tri_grams = trigrams(lines)
trigramProbs_smoothingBackoff = train_trigram(tri_grams,smoothing='backoff')
trigramProbs_smoothingLaplace = train_trigram(tri_grams)
trigramProbs_smoothingKneserNey = train_trigram(tri_grams,smoothing='kneser-ney')
triDiaryLaplace = [predictTrigram(trigramProbs_smoothingLaplace)[0] for _ in range(10)]
triDiaryBackoff = [predictTrigram(trigramProbs_smoothingBackoff)[0] for _ in range(10)]
triDiaryKneserNey = [predictTrigram(trigramProbs_smoothingKneserNey)[0] for _ in range(10)]

writeDiaryToFile("unigramDiary", unigramDiary)
writeDiaryToFile("bigramDiary", bigramDiary)
writeDiaryToFile("revBigramDiary", revBigramDiary)
writeDiaryToFile("biBigramDiary", biBigramDiary)
writeDiaryToFile("triDiaryLaplace", triDiaryLaplace)
writeDiaryToFile("triDiaryBackoff", triDiaryBackoff)
writeDiaryToFile("triDiaryKneserNey", triDiaryKneserNey)




15368
15368
15368
15368
15368


# **Smoothing Methods for Trigram Models**

## **1️⃣ Kneser-Ney Smoothing**
Kneser-Ney smoothing discounts **high-frequency trigrams** and redistributes probability to **lower n-grams**.

### **Formula**
$$
P_{KN}(w_i | w_{i-2}, w_{i-1}) = \frac{\max(C(w_{i-2}, w_{i-1}, w_i) - D, 0)}{C(w_{i-2}, w_{i-1})} + \lambda P(w_i | w_{i-1})
$$

**Where:**
$$
\begin{aligned}
&C(w_{i-2}, w_{i-1}, w_i) && \text{= Count of the trigram} \\
&C(w_{i-2}, w_{i-1}) && \text{= Count of the bigram} \\
&D && \text{= Discount constant (typically **0.75**)} \\
&P(w_i | w_{i-1}) && \text{= Backoff probability from the bigram model} \\
&\lambda && \text{= Normalization factor to adjust probabilities}
\end{aligned}
$$



### **Example Calculation**
#### **Given Data**
| **N-gram**             | **Count** |
|------------------------|----------|
| ("Aj", "subha", "uth") | 5        |
| ("subha", "uth")       | 10       |
| ("uth", "kar")         | 8        |
| ("kar", "nashta")      | 4        |

#### **Step 1: Apply Discount**
$$
P_{KN}("uth" | "Aj\ subha") = \frac{5 - 0.75}{10} + \lambda P("uth" | "subha")
$$

$$
= \frac{4.25}{10} + \lambda P("uth" | "subha")
$$

$$
= 0.425 + \lambda P("uth" | "subha")
$$

---

#### **Step 2: Compute Backoff Weight**
$$
\lambda = \frac{D}{C(w_{i-2}, w_{i-1})}
$$

$$
= \frac{0.75}{10} = 0.075
$$

$$
P_{KN}("uth" | "Aj\ subha") = 0.425 + (0.075 \times P("uth" | "subha"))
$$

---

## **2️⃣ Backoff Smoothing**
Backoff smoothing **falls back** to lower n-gram models when a trigram is missing.

### **Formula**
$$
P(w_i | w_{i-2}, w_{i-1}) =
\begin{cases} 
    P(w_i | w_{i-2}, w_{i-1}) & \text{if trigram exists} \\[5pt]
    \alpha P(w_i | w_{i-1}) + \beta P(w_i) & \text{otherwise}
\end{cases}
$$

**Where:**
$$
\alpha, \beta \text{ are weights ensuring probabilities sum to } 1.
$$


---

### **Example Calculation**
#### **Given Data**
| **N-gram**             | **Count** |
|------------------------|----------|
| ("Aj", "subha", "uth") | 0 (missing) |
| ("subha", "uth")       | 10       |
| ("uth")               | 20       |

#### **Step 1: Trigram Missing, Use Bigram**
$$
P("uth" | "Aj\ subha") = \alpha P("uth" | "subha") + \beta P("uth")
$$

---

#### **Step 2: Compute Bigram & Unigram Probabilities**
$$
P("uth" | "subha") = \frac{10}{20} = 0.5
$$

$$
P("uth") = \frac{20}{50} = 0.4
$$

---

#### **Step 3: Apply Backoff Formula**
$$
P("uth" | "Aj\ subha") = (0.7 \times 0.5) + (0.3 \times 0.4)
$$

$$
= 0.35 + 0.12 = 0.47
$$

---

## **🔹 Summary**
| **Method**       | **How It Works**                                    | **Best Used For**  |
|-----------------|----------------------------------------------------|------------------|
| **Kneser-Ney**  | Discounts frequent trigrams, redistributes probability | Large datasets  |
| **Backoff**     | Falls back to bigrams/unigrams if trigram is missing | Smaller datasets |

🚀 **Use Kneser-Ney for better accuracy, Backoff for simplicity!** 🔥 
---
Generated using AI
